In [1]:
import duckdb
import os

In [2]:
db_path = "/E2E Data Analysis Project/data/processed/ecommerce.duckdb"
con = duckdb.connect(db_path)

In [3]:
con.execute("""
CREATE OR REPLACE VIEW fact_order_items AS
SELECT * FROM read_parquet('/E2E Data Analysis Project/data/processed/fact_order_items.parquet');
""")

con.execute("SELECT COUNT(*) AS rows FROM fact_order_items;").fetchdf()

,rows
0,112650


In [4]:
from pathlib import Path

def run_sql_file(path: str):
    sql_text = Path(path).read_text(encoding="utf-8")
    return con.execute(sql_text).fetchdf()

monthly = run_sql_file("/E2E Data Analysis Project/sql/01_monthly_kpis.sql")
monthly.head()


,purchase_month,revenue,orders,items_sold,aov,mom_revenue_growth
0,2016-10,40325.11,265,313,152.170226,297.770912
1,2017-01,111798.36,750,913,149.064480,10255.730275
2,2017-02,234223.40,1653,1858,141.695947,1.095052
3,2017-03,359198.85,2546,2897,141.083602,0.533574
4,2017-04,340669.68,2303,2569,147.924307,-0.051585


In [5]:
top_categories = run_sql_file("/E2E Data Analysis Project/sql/02_top_categories.sql")
top_categories.head(10)


,product_category_name,revenue,orders,items_sold,revenue_share
0,beleza_saude,1233131.72,8647,9465,0.093267
1,relogios_presentes,1166176.98,5495,5859,0.088203
2,cama_mesa_banho,1023434.76,9272,10953,0.077407
3,esporte_lazer,954852.55,7530,8431,0.072220
4,informatica_acessorios,888724.61,6530,7644,0.067218
5,moveis_decoracao,711927.69,6307,8160,0.053846
6,utilidades_domesticas,615628.69,5743,6795,0.046563
7,cool_stuff,610204.10,3559,3718,0.046152
8,automotivo,578966.65,3810,4140,0.043790
9,brinquedos,471286.48,3804,4030,0.035645


In [6]:
pareto = run_sql_file("/E2E Data Analysis Project/sql/03_pareto_categories.sql")
pareto.head(20)


,product_category_name,revenue,revenue_share,cumulative_revenue_share,category_rank
0,beleza_saude,1233131.72,0.093267,0.093267,1
1,relogios_presentes,1166176.98,0.088203,0.181470,2
2,cama_mesa_banho,1023434.76,0.077407,0.258877,3
3,esporte_lazer,954852.55,0.072220,0.331097,4
4,informatica_acessorios,888724.61,0.067218,0.398315,5
5,moveis_decoracao,711927.69,0.053846,0.452161,6
6,utilidades_domesticas,615628.69,0.046563,0.498724,7
7,cool_stuff,610204.10,0.046152,0.544876,8
8,automotivo,578966.65,0.043790,0.588666,9
9,brinquedos,471286.48,0.035645,0.624312,10


In [7]:
n_80 = (pareto["cumulative_revenue_share"] <= 0.8).sum()
n_80

17

In [8]:
cancellations = run_sql_file("/E2E Data Analysis Project/sql/04_cancellations.sql")
cancellations.head()


,purchase_month,delivered_revenue,cancelled_revenue,cancelled_orders,cancelled_revenue_share
0,2016-09,134.97,59.50,1,0.440839
1,2016-10,40325.11,2992.67,12,0.074214
2,2016-12,10.90,0.00,0,0.000000
3,2017-01,111798.36,214.60,2,0.001920
4,2017-02,234223.40,2343.67,15,0.010006


In [9]:
states = run_sql_file("/E2E Data Analysis Project/sql/05_state_performance.sql")
states.head(10)


,customer_state,revenue,orders,revenue_share,cumulative_revenue_share
0,SP,5.067633e+06,40501,0.383287,0.383287
1,RJ,1.759651e+06,12350,0.133090,0.516378
2,MG,1.552482e+06,11354,0.117421,0.633799
3,RS,7.288975e+05,5345,0.055130,0.688928
4,PR,6.660635e+05,4923,0.050377,0.739306
5,SC,5.070121e+05,3546,0.038348,0.777653
6,BA,4.935841e+05,3256,0.037332,0.814985
7,DF,2.964984e+05,2080,0.022425,0.837411
8,GO,2.828367e+05,1957,0.021392,0.858803
9,ES,2.686434e+05,1995,0.020319,0.879121


In [10]:
mbr = run_sql_file("/E2E Data Analysis Project/sql/06_monthly_business_review.sql")
mbr.head(5)


,purchase_month,revenue,orders,items_sold,aov,mom_revenue_growth,cancelled_revenue,cancelled_orders,cancelled_revenue_share
0,2016-09,134.97,1,3,134.970000,NaN,59.50,1,0.440839
1,2016-10,40325.11,265,313,152.170226,297.770912,2992.67,12,0.074214
2,2016-12,10.90,1,1,10.900000,-0.999730,0.00,0,0.000000
3,2017-01,111798.36,750,913,149.064480,10255.730275,214.60,2,0.001920
4,2017-02,234223.40,1653,1858,141.695947,1.095052,2343.67,15,0.010006


In [11]:
mbr_clean = mbr[mbr["orders"] >= 100].copy()
mbr_clean.head(5)


,purchase_month,revenue,orders,items_sold,aov,mom_revenue_growth,cancelled_revenue,cancelled_orders,cancelled_revenue_share
1,2016-10,40325.11,265,313,152.170226,297.770912,2992.67,12,0.074214
3,2017-01,111798.36,750,913,149.064480,10255.730275,214.60,2,0.001920
4,2017-02,234223.40,1653,1858,141.695947,1.095052,2343.67,15,0.010006
5,2017-03,359198.85,2546,2897,141.083602,0.533574,6002.98,24,0.016712
6,2017-04,340669.68,2303,2569,147.924307,-0.051585,6084.25,14,0.017860


In [12]:
pareto = run_sql_file("/E2E Data Analysis Project/sql/03_pareto_categories.sql").copy()

# Ensure numeric
pareto["cumulative_revenue_share"] = pareto["cumulative_revenue_share"].astype(float)

# Minimum rank that reaches at least 80%
n_80_min = pareto.loc[pareto["cumulative_revenue_share"] >= 0.8, "category_rank"].min()
n_80_min



18